### Dataset支持特殊操作:Transformation 
 ####  1.一個Dataset可以透過Transformation變成另一個Dataset
 #### 2.通過transformation來做數據變化或pre-processing(預處理)
 #### 3.常用map() flat_map(),filter(),shuffle(),repeat()


# dataset.map()
#### map可以將map_func函數映射到數據集

#### 函數形式：flat_map(map_func，num_parallel_calls=None)
#### 參數map_func:映射函數
#### 參數num_parallel_calls：表示要並行處理的數字元素。如果未指定，將按順序處理元素。

In [ ]:
import tensorflow as tf
import numpy as np

a1 = np.array([1.0,2.0,3.0,4.0,5.0])
dataset = tf.data.Dataset.from_tensor_slices(a1)
dataset = dataset.map(lambda x: x*2) 
iterator = dataset.make_one_shot_iterator()

one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(len(a1)):
        print(sess.run(one_element))

# dataset.flat_map()

#### 與map不同是傳入數據必須是一個dataset object
#### 函數形式：flat_map(map_func)
#### 參數map_func: 映射函數

In [ ]:
import tensorflow as tf
import numpy as np

a2 = np.array([1.0,2.0,3.0,4.0,5.0])
dataset2 = tf.data.Dataset.from_tensor_slices(a2)

#### 若傳入函數不是一個object物件會報錯

In [4]:
dataset2 = dataset2.flat_map(lambda x: x+1)

TypeError: `map_func` must return a `Dataset` object.

#### 將函數包裝為Dataset object

In [5]:
#映射的函數-->要包裝成Dataset
dataset2 = dataset2.flat_map(lambda x:tf.data.Dataset.from_tensor_slices(x+[1]))

iterator2 = dataset2.make_one_shot_iterator()
one_element1 = iterator2.get_next()
with tf.Session() as sess:
    for i in range(len(a2)):
        print(sess.run(one_element1))

# dataset.zip()

In [ ]:
import tensorflow as tf
import numpy as np

#將給定的dataet壓縮再一起
dataset3 = tf.data.Dataset.from_tensor_slices(np.array([2.0,2.0,2.0]))
dataset4 = tf.data.Dataset.from_tensor_slices(np.array([3.0,3.0,3.0]))
dataset_zip = tf.data.Dataset.zip((dataset3,dataset4))
iterator3= dataset_zip.make_one_shot_iterator()
one_element2 = iterator3.get_next()

with tf.Session() as sess:
    for i in range(3): #顯示出0~2的元素
        print(sess.run(one_element2))

# dataset.concatenate()
#### 將兩個dataset進行串聯(合併)


In [2]:
import tensorflow as tf
import numpy as np

dataset_1 = tf.data.Dataset.from_tensor_slices([1.0,2.0,3.0])
dataset_2 = tf.data.Dataset.from_tensor_slices([4.0,5.0,6.0])

dataset_concat = dataset_1.concatenate(dataset_2)

iterator = dataset_concat.make_one_shot_iterator()
one_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(6):
     print(sess.run(one_element))

1.0
2.0
3.0
4.0
5.0
6.0


# dataset.filter()
#### 對傳入的Dataset進行條件過濾

In [6]:
import tensorflow as tf
import numpy as np

dataset = tf.data.Dataset.from_tensor_slices([1.0,2.0,3.0,4.0,5.0,6.0])
dataset = dataset.filter(lambda x: x>4)

iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(2):
     print(sess.run(one_element))

5.0
6.0


# dataset.generator()
#### 創建dataset, dataset元素由generator來生成

In [8]:
import tensorflow as tf
import numpy as np
#定義一個generator
def data_generator():
    dataset = np.array(range(9))
    for i in dataset:
        yield i
dataset = tf.data.Dataset.from_generator(data_generator,tf.int32)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(9):
     print(sess.run(one_element))

0
1
2
3
4
5
6
7
8


# dataset.barch()
#### 將數據集的連續元素堆疊成單個元素。

#### 函數形式：batch(batch_size,drop_remainder=False)

#### 參數batch_size:表示要在單個批次中合併的此數據集的連續元素個數。
#### 參數drop_remainder：表示在少於batch_size元素的情況下是否應刪除最後一批;默認是不刪除。

In [7]:
import tensorflow as tf
import numpy as np

"""
dataset.range(start,stop,step) 
#len(args=1)-->dataset.range(stop)  若無宣告:默認start=0 step=1
#len(args=2)-->dataset.range(start,stop) 
#len(args=3)-->dataset.range(start,stop,step)
"""
dataset1 = tf.data.Dataset.range(100)
dataset2 = tf.data.Dataset.range(0,-100,-1)
dataset = tf.data.Dataset.zip((dataset1,dataset2))
batched_dataset = dataset.batch(4)

iterator = batched_dataset.make_one_shot_iterator()
one_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(25):
     print(sess.run(one_element))

(array([0, 1, 2, 3], dtype=int64), array([ 0, -1, -2, -3], dtype=int64))
(array([4, 5, 6, 7], dtype=int64), array([-4, -5, -6, -7], dtype=int64))
(array([ 8,  9, 10, 11], dtype=int64), array([ -8,  -9, -10, -11], dtype=int64))
(array([12, 13, 14, 15], dtype=int64), array([-12, -13, -14, -15], dtype=int64))
(array([16, 17, 18, 19], dtype=int64), array([-16, -17, -18, -19], dtype=int64))
(array([20, 21, 22, 23], dtype=int64), array([-20, -21, -22, -23], dtype=int64))
(array([24, 25, 26, 27], dtype=int64), array([-24, -25, -26, -27], dtype=int64))
(array([28, 29, 30, 31], dtype=int64), array([-28, -29, -30, -31], dtype=int64))
(array([32, 33, 34, 35], dtype=int64), array([-32, -33, -34, -35], dtype=int64))
(array([36, 37, 38, 39], dtype=int64), array([-36, -37, -38, -39], dtype=int64))
(array([40, 41, 42, 43], dtype=int64), array([-40, -41, -42, -43], dtype=int64))
(array([44, 45, 46, 47], dtype=int64), array([-44, -45, -46, -47], dtype=int64))
(array([48, 49, 50, 51], dtype=int64), array

# padded_batch
#### 在batch的例子中,tensors大小是相同的,然後很多工作輸入數據大小並不一,為了解決這種情況,dataset.padded_batch()能將不同size的tensor進行batch,需要指定進行pad的維度
#### 將數據集的連續元素組合到填充批次中,此轉換將輸入數據集的多個連續元素組合為單個元素
#### 函數形式：padded_batch(batch_size,padded_shapes,padding_values=None,drop_remainder=False)

#### 參數batch_size：表示要在單個批次中合併的此數據集的連續元素數。
#### 參數padded_shapes：嵌套結構tf.TensorShape或tf.int64類似矢量張量的對象，表示batch之前應將每個輸入元素填充成相應的維度形狀。任何未知的維度（例如，tf.Dimension(None)在一個tf.TensorShape或-1類似張量的物體中）將被填充到每個batch中元素最大的維度尺寸。
#### 參數padding_values:(可選）標量形狀的嵌套結構tf.Tensor，表示用於各個組件的填充值。默認值0用於數字類型，空字符串用於字符串類型。
#### 參數drop_remainder:(可選）一個tf.bool標量tf.Tensor，表示在少於batch_size元素的情況下是否應刪除最後一批;默認行為是不刪除較小的批處理。

In [1]:
"""
註1:tf.cast(x, dtype, name=None)
此函数是类型转换函数

参数
x：imput
dtype：轉換目標類型
name：名稱
ex.
tensor `a` is [1.8, 2.2], dtype=tf.float
tf.cast(a, tf.int32) ==> [1, 2]  # dtype=tf.int32

註2:tf.fill（dims,value,name = None）
創建一個維度為dims,值為value的tensor對象．該操作會創建一個維度為dims的tensor對象，並將其值設置為value，該tensor對像中的值類型和value一致

當value為０時，該方法等同於tf.zeros()
當value為１時，該方法等同於tf.ones()
參數:
dims: 類型為int32的tensor對象，用於表示輸出的維度(1-D, nD)，通常為一個int32數組，如：[1], [2,3]等
value: 常量值(字符串，數字等)，該參數用於設置到最終返回的tensor對象值中
name: 當前操作別名(可選)
"""
import tensorflow as tf
import numpy as np

dataset = tf.data.Dataset.range(100)
#製造不同size的tensor
dataset = dataset.map(lambda x: tf.fill([tf.cast(x,tf.int32)],x)) #註1:tf.cast() 註2:tf.fill()
dataset = dataset.padded_batch(4,padded_shapes=[None])
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(10):
     print(sess.run(one_element))


[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]
[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]]
[[ 8  8  8  8  8  8  8  8  0  0  0]
 [ 9  9  9  9  9  9  9  9  9  0  0]
 [10 10 10 10 10 10 10 10 10 10  0]
 [11 11 11 11 11 11 11 11 11 11 11]]
[[12 12 12 12 12 12 12 12 12 12 12 12  0  0  0]
 [13 13 13 13 13 13 13 13 13 13 13 13 13  0  0]
 [14 14 14 14 14 14 14 14 14 14 14 14 14 14  0]
 [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]]
[[16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16  0  0  0]
 [17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17  0  0]
 [18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18  0]
 [19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19]]
[[20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20  0  0  0]
 [21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21  0  0]
 [22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22  0]
 [23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23]]
[[24 24 24 24 24 24 24 24 

# dataset.repeat()
#### 重複此數據集count次數

#### 函數形式：repeat(count=None)

#### 參數count:(可選）表示數據集應重複的次數。默認行為（如果count是None或-1）是無限期重複的數據集。

In [2]:
import tensorflow as tf
import numpy as np

dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])
dataset = dataset.repeat(3)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(18):
     print(sess.run(one_element))

1
2
3
4
5
6
1
2
3
4
5
6
1
2
3
4
5
6


# dataset.shuffle() 
#### 函數形式：shuffle(buffer_size,seed=None,reshuffle_each_iteration=None)
#### 參數buffer_size:表示新數據集將從中採樣的數據集中的元素數。
#### 參數seed:(可選）表示將用於創建分佈的隨機種子。
#### 參數reshuffle_each_iteration:(可選）一個布爾值，如果為true，則表示每次迭代時都應對數據集進行偽隨機重組。（默認為True。）
#### https://www.twblogs.net/a/5c0e196cbd9eee5e41834221 有關buffer size詳解

In [15]:
import tensorflow as tf
import numpy as np

dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6,7,8,9])

dataset=dataset.shuffle(buffer_size= 2)
dataset = dataset.batch(batch_size= 1)

iterator = dataset.make_one_shot_iterator()

element = iterator.get_next()

with tf.Session() as sess:
   for i in range(9):
       
       print(sess.run(element))

[1]
[3]
[2]
[4]
[6]
[7]
[8]
[5]
[9]


# dataset.skip()
#### 生成一個跳過count元素的數據集。

#### 函數形式：skip(count)

#### 參數count:表示應跳過以形成新數據集的此數據集的元素數。如果count大於此數據集的大小，則新數據集將不包含任何元素。如果count 為-1，則跳過整個數據集。

In [17]:
import tensorflow as tf
import numpy as np
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6,7,8,9])

#跳過前4個
dataset=dataset.skip(4)

iterator = dataset.make_one_shot_iterator()

element = iterator.get_next()

with tf.Session() as sess:
   for i in range(5):
       print(sess.run(element))

5
6
7
8
9


# dataset.take()
#### 提取前count個元素形成性數據集

#### 函數形式：take(count)

#### 參數count:表示應該用於形成新數據集的此數據集的元素數。如果count為-1，或者count大於此數據集的大小，則新數據集將包含此數據集的所有元素。

In [18]:
import tensorflow as tf
import numpy as np
dataset = tf.data.Dataset.from_tensor_slices([1,2,2,3,4,5,6,7,8,9])

#提取前5个元素形成新数据
dataset=dataset.take(5)

iterator = dataset.make_one_shot_iterator()

element = iterator.get_next()

with tf.Session() as sess:
   for i in range(5):
       print(sess.run(element))

1
2
2
3
4
